### Image Classifier

Before diving into model building, the dataset needs to be prepared for training, validation, and testing. <br />
First the Data Directory is split into train(90%) and test(10%) directories. <br />
Furthermore the training dataset is divided into train(80%) and validation(20%) set. <br />

The model itself is a CNN, as this is an image classification task. The first two layers of the model is dedicated for image preprocessing. <br />
Where augmentation (zoom, rotation, flip) is done to avoid overfitting and normalisation is done to standardise the images.<br />
Rest of the model follows standard image classification procedure, having three convolution layer and maxpool layers in between. <br />
To avoid further overfitting dropout is added. <br />

Another approach for building an image classifier is to use an already pretrained model. <br /> 
Using a pre-trained model like VGG16 or ResNet50 helps leverage the features learned from large-scale datasets like ImageNet.

I have implemented both approach to compare them and select the best one.


### Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Split The Data Directory Into Train and Test Directories

In [2]:
import os
import random
import shutil

data_path = 'Dataset' #change the directory name as needed
class_labels = ['Capacitor', 'IC', 'Resistor']

# path to destination folders
train_folder = os.path.join(data_path, 'train')
test_folder = os.path.join(data_path, 'test')

for label in class_labels:

    # Create a list of image filenames in 'data_path'
    imgs_list = [filename for filename in os.listdir(data_path + '/' + label) if os.path.splitext(filename)[-1] in ['.jpeg']]

    # Sets the random seed 
    random.seed(42)

    # Shuffle the list of image filenames
    random.shuffle(imgs_list)

    # determine the number of images for each set
    train_size = int(len(imgs_list) * 0.9)
    test_size = int(len(imgs_list) * 0.1)

    for folder_path in [train_folder + '/' + label, test_folder + '/' + label]:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    # Copy image files to destination folders
    for i, f in enumerate(imgs_list):
        if i < train_size:
            dest_folder = train_folder + '/' + label
        else:
            dest_folder = test_folder + '/' + label
        shutil.copy(os.path.join(data_path + '/' + label, f), os.path.join(dest_folder, f))


### Load The Dataset

In [3]:
import pathlib

test_dir = data_path + '/test'
test_dir = pathlib.Path(test_dir).with_suffix('')

train_dir = data_path + '/train'
train_dir = pathlib.Path(train_dir).with_suffix('')

### Create Train, Validation and Test Set

In [4]:
# define parameters
img_height = 640
img_width = 640

#train, validation and test set

train_ds = tf.keras.utils.image_dataset_from_directory(
train_dir,
validation_split=0.2,
subset="training",
seed=123,
image_size=(img_height, img_width),
batch_size=32)

val_ds = tf.keras.utils.image_dataset_from_directory(
train_dir,
validation_split=0.2,
subset="validation",
seed=123,
image_size=(img_height, img_width),
batch_size=32)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(640, 640),
    batch_size=32
)

Found 1845 files belonging to 3 classes.
Using 1476 files for training.
Found 1845 files belonging to 3 classes.
Using 369 files for validation.
Found 216 files belonging to 3 classes.


### Data Augmentation

To avoid overfitting random transformations(Flip, Rotation, Zoom) are done. This data augmentation layer will be added to the CNN model.

In [5]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

/Users/sumyea/Desktop/Job_Application/Evo/env/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Create The Model

The sequencial keras model consists of three 2D convolution layers, having a maxpool layer between each convolution layer. 

The images are normalized with a rescaling layer that standardizes the RGB channel values from [0, 255] to [0, 1]. 

A dropout layer is added to avoid overfitting followed by the fully connected layer with 128 units. 

ReLU activation function is used to activate the layers, but the output layer is activated with softmax.

In [6]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.5),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(3, activation='softmax')
])

### Compile The Model

As a good starting point adam optimizer and ctegorical cross entropy loss function is used to compile the model.

In [7]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Train The Model

Train the created model monitoring the validation loss for early stopping and save the best trained model.

In [165]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

epochs = 25
# Define the callbacks
model_path = 'models/'
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint(model_path+'best_model.keras',monitor='val_loss', 
                                   verbose=0, save_best_only=True,save_weights_only=False, mode='auto')
history = model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[early_stopping, model_checkpoint],
  epochs=epochs
)

Epoch 1/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 517s 11s/step - accuracy: 0.6902 - loss: 7.6161 - val_accuracy: 0.8537 - val_loss: 0.4343
Epoch 2/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 494s 10s/step - accuracy: 0.7858 - loss: 0.6591 - val_accuracy: 0.8916 - val_loss: 0.3038
Epoch 3/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 610s 13s/step - accuracy: 0.8597 - loss: 0.4034 - val_accuracy: 0.8943 - val_loss: 0.3105
Epoch 4/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 596s 13s/step - accuracy: 0.8933 - loss: 0.3065 - val_accuracy: 0.8889 - val_loss: 0.3163
Epoch 5/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 536s 11s/step - accuracy: 0.8958 - loss: 0.3089 - val_accuracy: 0.9079 - val_loss: 0.2795
Epoch 6/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 462s 10s/step - accuracy: 0.8572 - loss: 0.4201 - val_accuracy: 0.9214 - val_loss: 0.2535
Epoch 7/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 472s 10s/step - accuracy: 0.8867 - loss: 0.3014 - val_accuracy: 0.8943 - val_loss: 0.3111
Epoch 8/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 468s 10s/step - accuracy: 0.8868 - loss: 0.3065 - val_accuracy: 0.

### Evaluate The Model on Test Set

In [168]:
new_model = tf.keras.models.load_model(model_path+ 'best_model.keras')
eval_result = new_model.evaluate(test_ds)
print("[test loss, test accuracy]:", eval_result)

/Users/sumyea/Desktop/Job_Application/Evo/myenv/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9077 - loss: 0.3140
[test loss, test accuracy]: [0.30860400199890137, 0.9120370149612427]


### Pretrained Model

We can see we have achieved 91% accuracy with the trained model. To check if further accuracy can be achieved lets use an already trained model.

VGG16 is used as the base model, followed by an average pooling layer, a fully connected layer and a dropout layer.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(640, 640, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


### Compile The Model

Adam optimizer and ctegorical cross entropy loss function is used to compile the model.

In [155]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Train The Model

Train the created model monitoring the validation loss for early stopping and save the best trained model.

In [116]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

epochs = 25
# Define the callbacks
model_path = 'models/'
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint(model_path+'best_model_pretrained.keras',monitor='val_loss', 
                                   verbose=0, save_best_only=True,save_weights_only=False, mode='auto')
history = model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[early_stopping, model_checkpoint],
  epochs=epochs
)

Epoch 1/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2679s 57s/step - accuracy: 0.7145 - loss: 1.8726 - val_accuracy: 0.9485 - val_loss: 0.1743
Epoch 2/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2816s 60s/step - accuracy: 0.9297 - loss: 0.2306 - val_accuracy: 0.9593 - val_loss: 0.1069
Epoch 3/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2497s 53s/step - accuracy: 0.9541 - loss: 0.1271 - val_accuracy: 0.9675 - val_loss: 0.0872
Epoch 4/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2563s 55s/step - accuracy: 0.9658 - loss: 0.1032 - val_accuracy: 0.9756 - val_loss: 0.0716
Epoch 5/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2505s 53s/step - accuracy: 0.9740 - loss: 0.0763 - val_accuracy: 0.9783 - val_loss: 0.0714
Epoch 6/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2543s 54s/step - accuracy: 0.9796 - loss: 0.0638 - val_accuracy: 0.9810 - val_loss: 0.0648
Epoch 7/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2883s 62s/step - accuracy: 0.9821 - loss: 0.0544 - val_accuracy: 0.9810 - val_loss: 0.0582
Epoch 8/25
47/47 ━━━━━━━━━━━━━━━━━━━━ 2969s 63s/step - accuracy: 0.9898 - loss: 0.0416 - val_accu

### Evaluate The Model on Test Set

In [161]:
model.save(model_path+ 'best_model_pretrained.keras')
new_model = tf.keras.models.load_model(model_path+ 'best_model_pretrained.keras')
eval_result = model.evaluate(test_ds)
print("[test loss, test accuracy]:", eval_result)

/Users/sumyea/Desktop/Job_Application/Evo/myenv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


7/7 ━━━━━━━━━━━━━━━━━━━━ 280s 41s/step - accuracy: 0.9906 - loss: 0.0485
[test loss, test accuracy]: [0.10612954944372177, 0.9814814925193787]


It is evident the pretrained model performs better with 98% accuracy, this model will be deployed for the web app. <br />
As with the pretrained model I have already achieved a good accuracy, no further fine tuning is done. <br />
Although both models are included in the submission. 